In [1]:
import sys
sys.path.append('./utils')

import os

In [3]:
import cv2
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

In [4]:
# Global variables for mouse click
# ================== MOUSE diye click kore video theke coordinate fix korbo ===================================================
src_pts = []

board_state = [
    ["r","n","b","q","k","b","n","r"],
    ["p"*8],
    [" "*8],
    [" "*8],
    [" "*8],
    [" "*8],
    ["P"*8],
    ["R","N","B","K","Q","B","N","R"]
]

od = [
    [1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [1,1,1,1,1,1,1,1],
    [1,1,1,1,1,1,1,1]
]

In [5]:
prev_state = []
prev_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
# prev_fen = ""
cur_fen = ""
potential_fens = []

In [6]:
# ====================== BOX EXTRACTION ============================================================
def extract_squares(warped_img):
    squares = []
    square_size = warped_img.shape[0] // 8  # Assuming square board

    for i in range(8):
        row = []
        for j in range(8):
            x1 = j * square_size
            y1 = i * square_size
            x2 = (j + 1) * square_size
            y2 = (i + 1) * square_size
            square = warped_img[y1:y2, x1:x2]
            row.append(square)
        squares.append(row)
    return squares

In [8]:
# ================== OCCUPANCY DETECTOR MODEL ========================================================
model = tf.keras.models.load_model('cnn_model_OD.h5')

def is_occupied_cnn(square_img):
    resized = cv2.resize(square_img, (100, 100)) / 255.0
    resized = np.expand_dims(resized, axis=(0, -1))  # (1, 100, 100, 1)
    pred = model.predict(resized)
    print(pred)
    return pred[0][0] < 0.5  # If probability < 0.5, it's occupied


# ===================== OCCUPANCY DETECTOR ==========================
def has_changed(square1, square2, threshold=25):
    # diff = cv2.absdiff(square1, square2)
    # score = np.mean(diff)
    score1 = is_occupied_cnn(square1)
    score2 = is_occupied_cnn(square2)
    # return score > threshold
    # return score1 != score2, score1, score2

    if score1 and not score2: return 1 # move from
    elif not score1 and score2: return -1 # move to
    else: return 0 # no change

In [9]:
def get_indices(previous_squares,current_squares):
    move_from, move_to = None, None
    # Compare squares to detect changes
    for i in range(8):
        for j in range(8):
            compare = has_changed(previous_squares[i][j], current_squares[i][j])
            if compare == 0:
                continue
            else:
                if compare == 1:
                    move_from = (i,j)
                else:
                    move_to = (i, j)
    return move_from, move_to

In [10]:
# GENERATE LEGAL FENS

import chess

def generate_legal_fens(fen):
    board = chess.Board(fen)
    legal_fens = []

    for move in board.legal_moves:
        board.push(move)
        legal_fens.append(board.fen())
        board.pop()

    print("legal fens", legal_fens)

    return legal_fens


In [11]:
# UI INTEGRATION
from ChessViewer import ChessViewer
import importlib.util

white_player = "Player 1"
black_player = "Player 2"

def init_viewer(white_player_name="Player 1", black_player_name="Player 2"):
    white_player = white_player_name
    black_player = black_player_name

def view_game(fen_string):
    viewer = ChessViewer(fen_string, white_player, black_player)
    viewer.run()


In [12]:
# FEN MATCHER

import chess

def fen_to_occupancy_matrix(fen):
    piece_placement = fen.split(' ')[0]  # Only keep the piece layout part
    board = chess.Board()
    board.set_board_fen(piece_placement)
    
    occupancy = [[False for _ in range(8)] for _ in range(8)]

    for square in chess.SQUARES:
        row = 7 - (square // 8)
        col = square % 8
        if board.piece_at(square):
            occupancy[row][col] = True

    return occupancy

def fen_matcher(bm, cur_fen):
    fen_matrix = fen_to_occupancy_matrix(cur_fen)

    total = 64
    matches = 0

    for i in range(8):
        for j in range(8):
            if bm[i][j] == fen_matrix[i][j]:
                matches += 1

    accuracy = matches / total
    exact_match = matches == total
    return accuracy

In [13]:
def update_board_state_and_od(move_from, move_to):
    print(f"Move from: {move_from}; \t Move to: {move_to}")
    if move_from is None or move_to is None:
        print("gelo")
        return
    od[move_from[0]][move_from[1]]=0
    od[move_to[0]][move_to[1]]=1
    # p1 = board_state[move_from[0]][move_from[1]]
    # board_state[move_to[0]][move_to[1]] = p1
    # board_state[move_from[0]][move_from[1]] = " "

In [14]:
t = (1,2)
t[0]

1

# MAIN

In [15]:
import shutil

def transfer_file(from_, to_):
    
    # Source path and new destination path (with new name)
    # Move and rename the file
    shutil.move(from_, to_)
    
    print(f"File has been moved and renamed to {to_}")

In [16]:
# transfer_file("current_frame.jpg",f"moves/move_{1}.jpg")

In [17]:
# _, init_frame = cap.read()
init_warped = cv2.imread("initial_frame.jpg")
init_gray = cv2.cvtColor(init_warped, cv2.COLOR_BGR2GRAY)
# cv2.imwrite(, init_warped)
previous_squares = extract_squares(init_gray)

In [20]:
import cv2


move_count = 0

while True:
    if not os.path.exists("current_frame.jpg"):
        continue

    if os.path.exists("game_over.jpg"):
        break

    warped = cv2.imread("current_frame.jpg", cv2.IMREAD_GRAYSCALE)
    print("NICHHEEE")
    move_count+=1
    # transfer the file
    
    
    # gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    current_squares = extract_squares(warped)
    
    # move_detected = True
    # move_from, move_to = None, None
    move_from, move_to = get_indices(previous_squares,current_squares)

    prev_od = od.copy()
    update_board_state_and_od(move_from, move_to)
    print(prev_od)
    
    print("prev fen", prev_fen, "cur fen", cur_fen)
    if False:#prev_fen == "":
        cur_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
    else:
        print("HELLO")
        # prev_fen = cur_fen
        potential_fens = generate_legal_fens(prev_fen)
        print("potential fens", potential_fens)
        
        max_match = 0
        for pfen in potential_fens:
            if fen_matcher(od, pfen) > max_match:
                max_match = fen_matcher(od, pfen)
                cur_fen = pfen
    print("CUR FEN", cur_fen)
    # cur_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
    view_game(cur_fen)
    prev_fen = cur_fen

    transfer_file("current_frame.jpg",f"moves/move_{move_count}.jpg")
    
    move_text = f"Move: ({move_from[0]}, {move_from[1]}) -> ({move_to[0]}, {move_to[1]})"
    # cv2.putText(warped, move_text, (10, 790), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    print(move_text)

    # Update previous squares after move
    previous_squares = current_squares

NICHHEEE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[[1.7650202e-09]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[[1.2991997e-09]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[[1.13780525e-05]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[[0.91621417]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
[[5.4790346e-09]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[3.882254e-09]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[1.6149536e-06]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[1.1164476e-06]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[[1.1155233e-07]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[[1.1055447e-07]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
[[0.00013714]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
[[0.00013681]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
[[1.350497e-06]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
[[2.0874882e-06]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
[[0.9659193]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
[[0.96509826]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[[1.0740463e-05]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
[[7.0405367e-06]]

KeyboardInterrupt: 